In [6]:
import itertools

# Sample dataset
transactions = [
    ["Harry Potter", "Lord of the Rings", "Game of Thrones"],
    ["Harry Potter", "Percy Jackson"],
    ["Lord of the Rings", "Game of Thrones"],
    ["Harry Potter", "Lord of the Rings"],
    ["Percy Jackson", "Game of Thrones"],
    ["Harry Potter", "Game of Thrones"],
    ["Percy Jackson", "Harry Potter"],
    ["Game of Thrones", "Lord of the Rings"],
    ["Harry Potter", "Percy Jackson", "Lord of the Rings"],
    ["Game of Thrones"]
]

# Function to generate candidate itemsets of length k
def generate_candidates(itemsets, length):
    return set(itertools.combinations(itemsets, length))

# Function to filter out itemsets that meet the minimum support threshold
def filter_frequent_itemsets(transactions, candidates, min_support):
    # Step 1: Create a dictionary to count occurrences of each candidate itemset
    itemset_count = {}
    
    # Step 2: Loop through each transaction in the transactions list
    for transaction in transactions:
        # Step 3: Check each candidate itemset
        for candidate in candidates:
            # Step 4: If the candidate is part of the transaction, increase its count
            if set(candidate).issubset(transaction):
                if candidate in itemset_count:
                    itemset_count[candidate] += 1
                else:
                    itemset_count[candidate] = 1

    # Step 5: Calculate the total number of transactions
    num_transactions = len(transactions)
    
    # Step 6: Create a dictionary for itemsets that meet the minimum support
    frequent_itemsets = {}
    for itemset, count in itemset_count.items():
        support = count / num_transactions
        if support >= min_support:
            frequent_itemsets[itemset] = support
    
    # Step 7: Return the frequent itemsets
    return frequent_itemsets


# Function to generate association rules from frequent itemsets
def generate_association_rules(frequent_itemsets, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
        for i in range(1, len(itemset)):
            for subset in itertools.combinations(itemset, i):
                antecedent = set(subset)
                consequent = set(itemset) - antecedent
                if consequent:
                    support_antecedent = frequent_itemsets.get(tuple(sorted(antecedent)), 0)
                    support_itemset = frequent_itemsets[itemset]
                    confidence = support_itemset / support_antecedent if support_antecedent > 0 else 0
                    if confidence >= min_confidence:
                        lift = confidence / (frequent_itemsets.get(tuple(sorted(consequent)), 0) / len(transactions))
                        rules.append((antecedent, consequent, confidence, lift))
    return rules

# Step 1: Generate all itemsets
itemsets = set(item for transaction in transactions for item in transaction)

# Step 2: Initialize
min_support = 0.3
min_confidence = 0.6
frequent_itemsets = {}
k = 1

# Step 3: Generate frequent itemsets
while True:
    candidates = generate_candidates(itemsets, k)
    frequent_itemsets_k = filter_frequent_itemsets(transactions, candidates, min_support)
    if not frequent_itemsets_k:
        break
    frequent_itemsets.update(frequent_itemsets_k)
    k += 1

# Step 4: Generate association rules
rules = generate_association_rules(frequent_itemsets, min_confidence)

# Output results
print("Frequent Itemsets:")
for itemset, support in frequent_itemsets.items():
    print(f"Itemset: {itemset}, Support: {support:.2f}")

print("\nAssociation Rules:")
for antecedent, consequent, confidence, lift in rules:
    print(f"Rule: {antecedent} -> {consequent}, Confidence: {confidence:.2f}, Lift: {lift:.2f}")



Frequent Itemsets:
Itemset: ('Lord of the Rings',), Support: 0.50
Itemset: ('Game of Thrones',), Support: 0.60
Itemset: ('Harry Potter',), Support: 0.60
Itemset: ('Percy Jackson',), Support: 0.40
Itemset: ('Harry Potter', 'Lord of the Rings'), Support: 0.30
Itemset: ('Game of Thrones', 'Lord of the Rings'), Support: 0.30
Itemset: ('Harry Potter', 'Percy Jackson'), Support: 0.30

Association Rules:
Rule: {'Lord of the Rings'} -> {'Harry Potter'}, Confidence: 0.60, Lift: 10.00
Rule: {'Lord of the Rings'} -> {'Game of Thrones'}, Confidence: 0.60, Lift: 10.00
Rule: {'Percy Jackson'} -> {'Harry Potter'}, Confidence: 0.75, Lift: 12.50
